In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
import numpy as np
import random
import math
import time
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

cuda


In [6]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        #self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        self.fc = nn.Linear(enc_hid_dim *2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):     
        #src = [src len, batch size]
        #print('[ENCODER] encoder src: ', src.shape)
        
        embedded = self.dropout(self.embedding(src))
        #print('[ENCODER] embedded src: ', embedded.shape)
        
        outputs, hidden = self.rnn(embedded)
        #print('[ENCODER] outputs src: ', outputs.shape)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        #print('[ENCODER] hidden src: ', hidden.shape)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [7]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(enc_hid_dim+emb_dim, dec_hid_dim) 
        #self.rnn = nn.GRU(emb_dim, dec_hid_dim) 
        #self.fc_out = nn.Linear(dec_hid_dim, output_dim)
        self.fc_out = nn.Linear(dec_hid_dim*2+emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        input = input.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(input))
        #print('[DECODER] decoder embedded: {}'.format(embedded.shape))
        #print('[DECODER] decoder context: {}'.format(context.shape))
                
        #output, hidden = self.rnn(embedded, hidden.unsqueeze(0))
        rnn_input = torch.cat((embedded, context.unsqueeze(0)), dim = 2)
        #print('[DECODER] decoder rnn_input: {}'.format(rnn_input.shape))
        #print('[DECODER] decoder hidden: {}'.format(hidden.shape))
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        #print('[DECODER] decoder hidden: {}'.format(hidden.shape))
        assert (output == hidden).all()
        hidden = hidden.squeeze(0)
        
        output = torch.cat((embedded.squeeze(0), hidden, context), dim=1)
        #print('[DECODER] decoder output: {}'.format(output.shape))
        prediction = self.fc_out(output)
        
        return prediction, hidden

In [8]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #print('[SEQ2SEQ] src shape:', src.shape)
        #print('[SEQ2SEQ] trg shape:', trg.shape)
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        #print('[SEQ2SEQ] output shape:', outputs.shape)
        
        context = self.encoder(src)
        hidden = context
        #print('[SEQ2SEQ] encoder_outputs shape:', encoder_outputs.shape)
        #print('[SEQ2SEQ] hidden shape:', hidden.shape)
        
        input = trg[0,:]
        
        for t in range(1, trg_len):
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden = self.decoder(input, hidden, context)
            
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [9]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [10]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
            
print(model.apply(init_weights))

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('The model has {} trainable parameters'.format(count_parameters(model)))

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)
The model has 15927813 trainable parameters


In [11]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]


criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [12]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [13]:
N_EPOCHS = 30
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut-model_noAtt.pt')
    
    print('Epoch: {} | Time: {}m {}s'.format(epoch+1, epoch_mins, epoch_secs))
    print('\tTrain Loss: {} | Train PPL: {}'.format(train_loss, math.exp(train_loss)))
    print('\t Val. Loss: {} |  Val. PPL: {}'.format(valid_loss, math.exp(valid_loss)))

Epoch: 1 | Time: 0m 51s
	Train Loss: 5.1974907866658615 | Train PPL: 180.8179613304805
	 Val. Loss: 5.149212002754211 |  Val. PPL: 172.29566829811384
Epoch: 2 | Time: 0m 50s
	Train Loss: 4.748686240108003 | Train PPL: 115.43253423358165
	 Val. Loss: 5.210301399230957 |  Val. PPL: 183.14925091915524
Epoch: 3 | Time: 0m 50s
	Train Loss: 4.658639985559271 | Train PPL: 105.49251320951427
	 Val. Loss: 5.139114618301392 |  Val. PPL: 170.5646865967537
Epoch: 4 | Time: 0m 49s
	Train Loss: 4.604300916982642 | Train PPL: 99.91311086994024
	 Val. Loss: 5.131941020488739 |  Val. PPL: 169.34550233117324
Epoch: 5 | Time: 0m 49s
	Train Loss: 4.515908669795234 | Train PPL: 91.46063579237449
	 Val. Loss: 5.196409702301025 |  Val. PPL: 180.6225874864371
Epoch: 6 | Time: 0m 49s
	Train Loss: 4.495446869980396 | Train PPL: 89.60820325545453
	 Val. Loss: 5.160942852497101 |  Val. PPL: 174.3287444348803
Epoch: 7 | Time: 0m 49s
	Train Loss: 4.4838172045048115 | Train PPL: 88.57212612031996
	 Val. Loss: 5.3524

In [14]:
model.load_state_dict(torch.load('tut-model_noAtt.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print('| Test Loss: {} | Test PPL: {} |'.format(test_loss, math.exp(test_loss)))

| Test Loss: 4.061550289392471 | Test PPL: 58.06425774212494 |


In [15]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        hidden = model.encoder(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    context = hidden
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, context)
            #hidden = hidden.squeeze(0)
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

In [16]:
example_idx = 12

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print('src = {}'.format(src))
print('trg = {}'.format(trg))

src = ['ein', 'schwarzer', 'hund', 'und', 'ein', 'gefleckter', 'hund', 'kämpfen', '.']
trg = ['a', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'are', 'fighting']


In [17]:
translation = translate_sentence(src, SRC, TRG, model, device)

print('predicted trg = {}'.format(translation))

predicted trg = ['a', 'black', 'and', 'black', 'dog', 'dog', 'dog', 'are', 'running', 'a', 'a', '.', '<eos>']


In [18]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [19]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print('BLEU score = {}'.format(bleu_score*100))

BLEU score = 12.77655106449502
